# Envoyer un CSV dans Big Querry

**Auteur :**  Steve Caron  
**Date de création :** 2023/07/25  
**Présentation :** Ce notebook permet de charger le contenu d'un CSV stocké dans un bucket dans une table BigQuery. Il se lance depuis un dataproc appartenant au projet GCP.


**Prérequis :** 
* Un bucket gcp contenant les données au format csv. (BUCKET_NAME)  
* Une table bigQuery Dans le même projet (TABLE_ID)  

**Inputs :**  
* fichier d'entré: liste_station.csv  Télechargeable ici : https://www.data.gouv.fr/fr/datasets/liste-des-stations-en-open-data-du-reseau-meteorologique-infoclimat-static-et-meteo-france-synop/

**Params:** 
* BUCKET_NAME: nom du bucket contenant mes fichier source. Doit être dans le même projet
* INPUT_FILE_NAME: nom du fichier a traiter. Doit être au format csv
* TABLE_ID: ID de la table dans BigQuerry. Doit être dans le même projet
* PREMIERE_LIGNE : Paramètre permettant de dire si il faut charger la première ligne du CSV dans la table. OUI = 1, NON = 0
* AUTODETECTION : Paramètre du job config BigQuery, permettant d'activer ou non la detection automatique du schema (Activée = True, Désactivée = False)

In [ ]:
from google.cloud import bigquery, storage
import os

In [ ]:
BUCKET_NAME = "code_de_source_lake"
INPUT_FILE_NAME="liste_station.csv"
TABLE_ID = "code-de-source.donnees_code_de_source.stations_meteo"
PREMIERE_LIGNE = 1
AUTODETECTION = True



#Je creer le chemin de mon fichier
chemin_fichier_dataproc = f"Data/{INPUT_FILE_NAME}"

# Telechargement du fichier depuis un bucket

In [ ]:
# J'instancie un client storage
storage_client = storage.Client()
# Je précise le nom de bucket
bucket = storage_client.bucket(BUCKET_NAME)
#Dans un BLOB nommé mon_blob je charge le contenu de mon fichier
mon_blob = bucket.blob(INPUT_FILE_NAME)


# Je télecharge ensuite le contenu du BLOB nommé mon_blob dans un fichier
mon_blob.download_to_filename(chemin_fichier_dataproc)
print(f"Fichier CSV téléchargé depuis le bucket {BUCKET_NAME} vers {INPUT_FILE_NAME}.")

# Envoie vers BigTable

In [ ]:
# J'instancie un Client BigQuerry
client = bigquery.Client()

# J'ouvre mon fichier qui est situé dans mon environnement de travail
with open(chemin_fichier_dataproc, "rb") as source_file:
    # Je creer une configuration de JOB en précisant le format et en sautant la premiere ligne  (header)
    job_config = bigquery.LoadJobConfig(
        autodetect= AUTODETECTION,
        skip_leading_rows= PREMIERE_LIGNE,
        source_format=bigquery.SourceFormat.CSV,
    )
    # Je lance mon job qui va envoyer le contenu de mon fichier source dans une table BigQuerry
    job = client.load_table_from_file(source_file, TABLE_ID, job_config=job_config)

job.result()

print(f"Chargement du fichier {INPUT_FILE_NAME} dans la table {TABLE_ID} terminé.")

# Suppression du fichier

In [ ]:
# Je vérifie si le fichier existe dans le dataproc
if os.path.exists(chemin_fichier_dataproc):
    # Si il existe, alors je le supprime
    os.remove(chemin_fichier_dataproc)
    print(f"Le fichier{INPUT_FILE_NAME} a été correctement supprimé")
else:
    # Si il n'existe pas j'affiche le message suivant
    print(f"Impossible de supprimer le fichier {INPUT_FILE_NAME} car il n'existe pas")